In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
from preprocessing import my_plot, analysis, generate_mean_std, SPCRules

In [3]:
def xlsx2csv(filename):
    prefix, postfix = filename.split('.')
    df = pd.read_excel(f"./csv/{filename}")
    df.columns = df.iloc[0].to_numpy()
    df = df.iloc[1:]
    df.to_csv(f"./csv/{prefix}.csv", index = False)
    return

def save_csv(filename: str, df: pd.DataFrame):
    df.to_csv(f"csv/{filename}", index = False)
    return

def SWaT_generate_mean_std(df, target_features, label_features, wnd_size=5000):
    df_fe = df[target_features].copy()
    for target in target_features:
        op = "mean"
        df_fe[f"{target}_{op}_{wnd_size}"] = df_fe[target].rolling(wnd_size, min_periods = 200).mean()
    
    for target in target_features:
        op = "std"
        df_fe[f"{target}_{op}_{wnd_size}"] = df_fe[target].rolling(wnd_size, min_periods = 200).std()
    df_fe[label_features] = df[label_features]
    df_fe.dropna(axis = 0, inplace = True)
    return df_fe

# Generate spc labels by 6 rules
class SPCRules(object):
    def __init__(self, data, mean, std):
        super().__init__()
        self.data = data
        self.n = len(data)
        self.index = np.array(range(self.n))
        self.mean = mean
        self.std = std
        self.upper_c = mean + std
        self.upper_b = mean + 2 * std
        self.upper_a = mean + 3 * std
        self.lower_c = mean - std
        self.lower_b = mean - 2 * std
        self.lower_a = mean - 3 * std
#         print(self.data.shape, self.mean.shape, self.std.shape, self.upper_c.shape, self.lower_c.shape)
    
    def detect(self, idx: int):
        if idx == 1:
            # One point beyond the 3 σ control limit
            out = (self.data < self.lower_a) | (self.data > self.upper_a)
        elif idx == 2:
            # Nine or more points on one side of the centerline without crossing
            counter = 9
            upside = (self.data > self.mean).rolling(counter).sum()
            downside = (self.data < self.mean).rolling(counter).sum()
            out = (upside >= counter) | (downside >= counter)
        elif idx == 3:
            # Two out of three points in zone A or beyond
            counter = 3
            upside = (self.data > self.upper_b).rolling(counter).sum()
            downside = (self.data < self.lower_b).rolling(counter).sum()
            counter -= 1
            out = (upside >= counter) | (downside >= counter)
        elif idx == 4:
            # Four out of five points in zone B or beyond
            counter = 5
            upside = (self.data > self.upper_c).rolling(counter).sum()
            downside = (self.data < self.lower_c).rolling(counter).sum()
            counter -= 1
            out = (upside >= counter) | (downside >= counter)
        elif idx == 5:
            # Fifteen points are all in zone c
            counter = 15
            out = (self.data < self.upper_c) & (self.data > self.lower_c)
            out = out.rolling(counter).sum()
            out = (out == counter)
        elif idx == 6:
            # Eight continual points with none in zone c
            counter = 8
            out = (self.data > self.upper_c) | (self.data < self.lower_c)
            out = out.rolling(counter).sum()
            out = (out == counter)
        else:
            raise ValueError("Only implement rule 1~6")
        return out
    
    # Six or more points are continually increasing or decreasing
    def rule7(self):
        pass
    
    # Fourteen or more points alternate in direction
    def rule8(self):
        ofc8_ind = []
        for i in range(self.n - 13):
            d = self.data[i:i+14]
            idx = self.index[i:i+14]
            diff = list(v - u for u, v in zip(d, d[1:]))
            if all(u * v < 0):
                pass

In [ ]:
# transform xlsx to csv (csv is faster than xlsx)
filename = "SWaT_Dataset_Normal_v1.xlsx"
xlsx2csv(filename)
filename = "SWaT_Dataset_Attack_v0.xlsx"
xlsx2csv(filename)

In [ ]:
filename = "SWaT_Dataset_Normal_v1.csv"
normal = pd.read_csv(f"csv/{filename}")

filename = "SWaT_Dataset_Attack_v0.csv"
anomaly = pd.read_csv(f"csv/{filename}")

In [ ]:
normal

In [ ]:
anomaly.head()

In [ ]:
# have wrong value: "A ttack"
print(np.unique(normal["Normal/Attack"]), np.unique(anomaly["Normal/Attack"]))
# anomaly["Normal/Attack"] = anomaly["Normal/Attack"].map({'A ttack': "Attack", "Attack": "Attack", "Normal": "Normal"})

In [ ]:
# Combine normal and anomaly, because their timestamp is continual
normal.columns = anomaly.columns # their columns have weird difference
total = pd.concat([normal, anomaly], axis = 0)
total
save_csv("SWaT_total.csv", total)

# Feature engineering
- Normal(0: 496754), Anomaly(496754: 940190)
- "Normal": 0, "Attack": 1

In [ ]:
filename = "SWaT_total.csv"
total = pd.read_csv(f"csv/{filename}")
total

In [ ]:
target = "Normal/Attack"
total[target] = total[target].map({"Normal": 0, "Attack": 1})

In [ ]:
target_features = [
    'FIT101', 'LIT101', 'MV101', 'P101', 'P102', 'AIT201',
    'AIT202', 'AIT203', 'FIT201', 'MV201', 'P201', 'P202', 'P203',
    'P204', 'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301',
    'MV302', 'MV303', 'MV304', 'P301', 'P302', 'AIT401', 'AIT402',
    'FIT401', 'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401', 'AIT501',
    'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504',
    'P501', 'P502', 'PIT501', 'PIT502', 'PIT503', 'FIT601', 'P601', 'P602',
    'P603'
]

## Remove dummy features

In [ ]:
# find dummy features
op = "std"
win = 5000
lst = []
for feature in target_features:
    num = len(np.unique(df[f"{feature}_{op}_{win}"]))
    if num < 10:
        lst.append(feature)
        
print(lst)
total.drop(columns = lst, inplace = True)

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
# 496754: 940190 是異常訊號，將前後共 500 row 加進來 (496254, 940690)
target_points = total.iloc[496254: 940690]
X = target_points.drop(columns = ["Timestamp", "Normal/Attack"])
Y = target_points["Normal/Attack"]
selector = SelectKBest(f_classif, k = 30)
X_new = selector.fit_transform(X, Y)
lst = selector.get_feature_names_out(list(X.columns))
print(f"Select {len(lst)} features by p_val")
lst = list(lst)
print(lst)
lst = ["Timestamp"] + lst + ["Normal/Attack"]
total = total[lst]

In [ ]:
total

## Data synthesis

In [ ]:
target_features = [
    'FIT101', 'LIT101', 'MV101', 'P101', 'AIT203', 'FIT201',
    'MV201', 'P203', 'P205', 'DPIT301', 'FIT301', 'LIT301', 'MV302',
    'MV304', 'P302', 'AIT402', 'FIT401', 'LIT401', 'P402', 'UV401',
    'AIT501', 'AIT502', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'P501',
    'PIT501', 'PIT502', 'PIT503'
]
label_features = ['Timestamp', 'Normal/Attack']
df = SWaT_generate_mean_std(total, target_features, label_features, wnd_size = 1000)
df.to_csv("csv_fe/SWaT_mean_std.csv", index = False)

In [11]:
filename = "SWaT_mean_std.csv"
df = pd.read_csv(f"csv_fe/{filename}")
features_num = 30
features_name = df.columns[:features_num]
datas = df.iloc[:, :features_num]
datas.columns = ["" for _ in range(datas.shape[1])]
means = df.iloc[:, features_num: 2 * features_num]
means.columns = ["" for _ in range(means.shape[1])]
stds = df.iloc[:, 2 * features_num: 3 * features_num]
stds.columns = ["" for _ in range(stds.shape[1])]

rule_gen = SPCRules(datas, means, stds)
rule_lst = []
for i in range(1, 7):
    rule_df = rule_gen.detect(idx = i)
    rule_df.columns = [f"{name}_rule{i}" for name in features_name]
    rule_lst.append(rule_df)

df_fe = pd.concat([df.iloc[:, :features_num]] + rule_lst, axis = 1)
#     df_fe['fault_label'] = df['fault_label']
df_fe['anomaly_label'] = df['Normal/Attack']
df_fe['Timestamp'] = df['Timestamp']
# remove dummy rule columns
# remove_cols = ['LC51_03CV_rule2', 'LC51_03X_rule2', 'P51_06_rule2', 'T51_01_rule2', 'F51_01_rule2', 'P57_03_rule2', 'FC57_03PV_rule2', 'FC57_03CV_rule2', 'FC57_03X_rule2', 'T51_01_rule4', 'P57_03_rule4', 'FC57_03PV_rule4', 'FC57_03CV_rule4', 'FC57_03X_rule4', 'T51_01_rule5', 'T51_01_rule6', 'FC57_03PV_rule6']
# df_fe.drop(columns = remove_cols, inplace = True)
df_fe.to_csv(f"csv_fe/SWaT_spc_label.csv", index = False)

/tmp/ipykernel_11010/3710321439.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fe['anomaly_label'] = df['Normal/Attack']
/tmp/ipykernel_11010/3710321439.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fe['Timestamp'] = df['Timestamp']


In [12]:
for feature in df_fe.columns:
    print(feature)

FIT101
LIT101
MV101
P101
AIT203
FIT201
MV201
P203
P205
DPIT301
FIT301
LIT301
MV302
MV304
P302
AIT402
FIT401
LIT401
P402
UV401
AIT501
AIT502
FIT501
FIT502
FIT503
FIT504
P501
PIT501
PIT502
PIT503
FIT101_rule1
LIT101_rule1
MV101_rule1
P101_rule1
AIT203_rule1
FIT201_rule1
MV201_rule1
P203_rule1
P205_rule1
DPIT301_rule1
FIT301_rule1
LIT301_rule1
MV302_rule1
MV304_rule1
P302_rule1
AIT402_rule1
FIT401_rule1
LIT401_rule1
P402_rule1
UV401_rule1
AIT501_rule1
AIT502_rule1
FIT501_rule1
FIT502_rule1
FIT503_rule1
FIT504_rule1
P501_rule1
PIT501_rule1
PIT502_rule1
PIT503_rule1
FIT101_rule2
LIT101_rule2
MV101_rule2
P101_rule2
AIT203_rule2
FIT201_rule2
MV201_rule2
P203_rule2
P205_rule2
DPIT301_rule2
FIT301_rule2
LIT301_rule2
MV302_rule2
MV304_rule2
P302_rule2
AIT402_rule2
FIT401_rule2
LIT401_rule2
P402_rule2
UV401_rule2
AIT501_rule2
AIT502_rule2
FIT501_rule2
FIT502_rule2
FIT503_rule2
FIT504_rule2
P501_rule2
PIT501_rule2
PIT502_rule2
PIT503_rule2
FIT101_rule3
LIT101_rule3
MV101_rule3
P101_rule3
AIT203_ru